In [4]:
import numpy as np
#We used object oriented programmation for this project
class ising:

    def __init__(self, theta = (0, 0.2), size = 10, grid = None):
        self.alpha = theta[0]
        self.beta = theta[1]
        self.size = size
        if grid is None:
            #Création of grid
            self.grid = np.random.choice([-1, 1], size=(size, size), replace=True)
        else:
            self.grid = grid
            
#Method to compute the neighbours of one coordinate
    def neighbours(self, i, j) :
        l = []
        if i <= self.size -1 and j <= self.size - 1 :
            for k in [-1,1] :
                if i - k >= 0 and i - k <= (self.size - 1):
                    l.append((i-k,j))
                if j - k >= 0 and j - k <= (self.size - 1) :
                    l.append((i,j - k))
            if i == 0 :
                l.append(((self.size -1),j))
            if i == self.size -1 :
                l.append((0,j))
            if j == 0 :
                 l.append((i,(self.size -1)))
            if j == self.size -1 :
                l.append((i,0))
            return l
        else :
             raise Exception('OUT OF GRID')
         

    def sum_neighbours(self, i, j):
        l = []
        for x in self.neighbours(i, j):
            l.append(self.grid[x[0],x[1]])
        return sum(l)
    

    def spin_conditional_prob(self, i, j):
        e = np.exp(-(2 * (self.alpha + self.beta*self.sum_neighbours(i,j))))
        return 1 / (1 + e)
#Gibbs sampler    
    def gibbs_sampling(self, N_iter):
        grid = np.copy(self.grid)
        mag_field = [np.sum(grid)]
        for _ in range(N_iter):
            random_index = np.random.randint(0, 10, size=2)
            p = self.spin_conditional_prob(random_index[0],random_index[1])
            u = np.random.uniform(0,1)
            if u <= p:
                grid[random_index[0], random_index[1]] = 1
            else:
                grid[random_index[0], random_index[1]] = -1
            mag_field.append(np.sum(grid))
        return grid,mag_field
                    
    def likelihood(self):
            s = 0
            for i in range(self.size):
                for j in range(self.size):
                    neighbours = self.neighbours(i, j)
                    s+= sum(self.grid[i, j]*self.grid[neighbours[k][0], neighbours[k][1]] for k in range(len(neighbours)))
            return np.exp(self.alpha*sum(self.grid[i, j] for i in range(self.size) for j in range (self.size)) + self.beta * s)
        
    
    def exchange_algorithm(self, beta, N_iter, N_gibbs):

        potential_final_beta = beta
        beta_sample = []

        for _ in range(N_iter):

            # We generate a beta proposal using a gaussian perturbation
            candidate_beta = np.random.normal(potential_final_beta, 0.1)
            if candidate_beta >=0 and candidate_beta <= 1 :

                # Then we approximate w thanks to Gibbs sampling
                model = ising(theta = (0, candidate_beta), size = self.size, grid = self.grid)
                w = model.gibbs_sampling(N_gibbs)[0]

                # We now compute a
                f_y_potential = ising(theta = (0, potential_final_beta), size = self.size, grid = self.grid)
                f_y_candidate = ising(theta = (0, candidate_beta), size = self.size, grid = self.grid)
                f_w_potential = ising(theta = (0, potential_final_beta), size = self.size, grid = w)
                f_w_candidate = ising(theta = (0, candidate_beta), size = self.size, grid = w)
                a =  min(1,(np.random.normal(candidate_beta, 0.1) * f_y_candidate.likelihood() * f_w_potential.likelihood()) / (np.random.normal(potential_final_beta, 0.1) * f_y_potential.likelihood() * f_w_candidate.likelihood()))
                
                # We now accept/reject
                r = np.random.uniform(0, 1)
                if r < a:
                    potential_final_beta = candidate_beta
                    f_y_potential = f_y_candidate
            beta_sample.append(potential_final_beta)
        return potential_final_beta  , beta_sample 
        
    
        


In [ ]:
#Creation of an ising grid with the parameters of the paper
isi = ising(theta = (0,0.2))

isi.grid


In [ ]:
#Execution of the gibbs sampler
isi2 = isi.gibbs_sampling(50000)

isi2[0]

In [ ]:
#Execution of the exchange algorithm 
isi3 = isi.exchange_algorithm(0.2,100000,5000)

In [ ]:
import matplotlib.pyplot as plt
#We compute the autocorrelations of the gibbs sampler and the exchange algorithm to check if they converge towards zero
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]


plt.plot(autocorr(isi2[1])/ (autocorr(isi2[1])[0]))

In [ ]:
plt.plot(autocorr(isi3[1]) / (autocorr(isi3[1])[0]))

In [148]:
#Function to plot the evolution of means of the betas with the number of iterations
moy = []
for k in range(len(isi3[1])) :
    l = []
    for i in range(k+1) :
        l.append(isi3[1][i])
    moy.append(sum(l)/len(l))

In [ ]:
#We plot the evolution of the mean of the betas to check if it converges (it converges towards a value such that the mean of the posterior converges towards 0.2)
plt.plot(range(len(moy)),moy)

In [ ]:
#Note : since the algorithm take a lot of time to execute (for example, the exchange can take multiple hours depending of the parameters), the plots are on a different document